In [13]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error as MSE
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as MSE
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [14]:
np.random.seed(42)

**1. Import the train datasets**

In [15]:
train_data=pd.read_csv('Data-Train.csv', index_col=0)
test_data=pd.read_csv('Data-Test.csv', index_col=0)

catvar = ['productID', 'brandID', 'attribute1', 'attribute2', 'attribute3', 'weekday'] 

encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

transformed_columns = encoder.fit_transform(train_data[catvar])

encoded_dataset = pd.DataFrame(transformed_columns, columns=encoder.get_feature_names_out(catvar))

train_data = train_data.drop(columns=catvar)

train_data = pd.concat([train_data, encoded_dataset], axis=1)
train_data["log-sales"]=np.log(train_data.sales)

Y_train=train_data["log-sales"]
X_train=train_data.drop('log-sales',axis=1)

D:\anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [16]:
Model_1 = DecisionTreeRegressor()

2. Setup Pipeline

In [17]:
pca=PCA()
RG = Model_1
pipe = Pipeline(steps=[("pca",pca),("RG",RG)])

param_grid = {
    "pca__n_components": [5, 10, 30, 45, 60, 100 ,150 ,170],
    "RG__ccp_alpha":[0,0.001,0.005,0.1,0.5,2,8]
}

3. Setup GridSearch

In [18]:
score_metric = 'neg_mean_squared_error'
search = GridSearchCV(pipe,param_grid,
                      n_jobs=-1,
                      cv=10,
                      scoring=score_metric)
search.fit(X_train,Y_train)

D:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
70 fits failed out of a total of 560.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
70 fits failed with the following error:
Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\anaconda\lib\site-packages\sklearn\base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "D:\anaconda\lib\site-packages\sklearn\pipeline.py", line 423, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "D:\anaconda\lib\site-packages\sklearn\pipeline.py", line 377, in _fit
    X, 

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('pca', PCA()),
                                       ('RG', DecisionTreeRegressor())]),
             n_jobs=-1,
             param_grid={'RG__ccp_alpha': [0, 0.001, 0.005, 0.1, 0.5, 2, 8],
                         'pca__n_components': [5, 10, 30, 45, 60, 100, 150,
                                               170]},
             scoring='neg_mean_squared_error')

In [19]:
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV score=-0.218):
{'RG__ccp_alpha': 0, 'pca__n_components': 5}


In [20]:
df = pd.DataFrame(search.cv_results_)
df = df.sort_values(by=['rank_test_score'])
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_RG__ccp_alpha,param_pca__n_components,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.099672,0.014422,0.006382,0.002509,0,5,"{'RG__ccp_alpha': 0, 'pca__n_components': 5}",-0.216379,-0.201034,-0.195603,-0.193851,-0.240695,-0.227110,-0.156434,-0.308947,-0.226945,-0.216711,-0.218371,0.037570,1
8,0.079057,0.008939,0.007053,0.003879,0.001,5,"{'RG__ccp_alpha': 0.001, 'pca__n_components': 5}",-0.292515,-0.249703,-0.255467,-0.248069,-0.229564,-0.231652,-0.185269,-0.219830,-0.228672,-0.249109,-0.238985,0.026341,2
1,0.128324,0.026587,0.008360,0.005797,0,10,"{'RG__ccp_alpha': 0, 'pca__n_components': 10}",-0.234915,-0.244596,-0.225580,-0.289208,-0.261176,-0.284832,-0.220442,-0.213109,-0.281923,-0.188490,-0.244427,0.032335,3
9,0.130369,0.021682,0.005732,0.004501,0.001,10,"{'RG__ccp_alpha': 0.001, 'pca__n_components': 10}",-0.261026,-0.255467,-0.244952,-0.272522,-0.259587,-0.312277,-0.195571,-0.266768,-0.282080,-0.271707,-0.262196,0.028127,4
10,0.264652,0.081851,0.006400,0.002653,0.001,30,"{'RG__ccp_alpha': 0.001, 'pca__n_components': 30}",-0.296788,-0.310261,-0.247941,-0.292526,-0.275321,-0.342810,-0.321846,-0.394898,-0.338602,-0.317482,-0.313848,0.038320,5
12,0.435489,0.053556,0.006904,0.001700,0.001,60,"{'RG__ccp_alpha': 0.001, 'pca__n_components': 60}",-0.287934,-0.302317,-0.335903,-0.344083,-0.276844,-0.385973,-0.277497,-0.279122,-0.293283,-0.371176,-0.315413,0.038715,6
11,0.350630,0.109999,0.007200,0.001600,0.001,45,"{'RG__ccp_alpha': 0.001, 'pca__n_components': 45}",-0.325341,-0.339483,-0.260562,-0.321818,-0.260943,-0.339849,-0.318368,-0.395699,-0.329442,-0.345482,-0.323699,0.037631,7
16,0.092723,0.025915,0.012108,0.011980,0.005,5,"{'RG__ccp_alpha': 0.005, 'pca__n_components': 5}",-0.370687,-0.305004,-0.329524,-0.342721,-0.317568,-0.342989,-0.272509,-0.381065,-0.320862,-0.308787,-0.329172,0.030340,8
3,0.305810,0.016325,0.006304,0.004145,0,45,"{'RG__ccp_alpha': 0, 'pca__n_components': 45}",-0.341913,-0.331286,-0.278723,-0.303806,-0.258874,-0.421854,-0.326227,-0.278307,-0.387202,-0.380425,-0.330862,0.050447,9
19,0.364827,0.068431,0.006400,0.004800,0.005,45,"{'RG__ccp_alpha': 0.005, 'pca__n_components': 45}",-0.377052,-0.301313,-0.338619,-0.335048,-0.325580,-0.354093,-0.321955,-0.311695,-0.368280,-0.323550,-0.335719,0.023057,10


4. Best Hyperparameter

In [21]:
parms = pd.DataFrame(df['params'])
for i in parms.iloc[0]:
    print(i)

{'RG__ccp_alpha': 0, 'pca__n_components': 5}


5. Evaluate Model Log(Sales)


In [22]:

# Clean and preprocess data
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
Y_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(X_train.mean(), inplace=True)
Y_train.fillna(Y_train.mean(), inplace=True)

# Normalize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Extract the best hyperparameters
best_pca = parms.iloc[0][0]['pca__n_components']
best_ccp_alpha = parms.iloc[0][0]['RG__ccp_alpha']

# Prepare the PCA and the Decision Tree Regressor with the best parameters
pca = PCA(n_components=30)
Model_2 = DecisionTreeRegressor(ccp_alpha=0)

# Prepare the k-fold cross-validation configuration
k = 10  # Number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=132)

# Lists to store the MSE for each fold
mse_scaled_values = []
mse_true_values = []

for train_index, test_index in kf.split(X_train_scaled):
    # Split the data into training and validation folds
    X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[test_index]
    Y_train_fold, Y_val_fold = Y_train.iloc[train_index], Y_train.iloc[test_index]
    
    # Apply PCA transformation
    PX_train_fold = pca.fit_transform(X_train_fold)
    PX_val_fold = pca.transform(X_val_fold)
    
    # Fit the model on the training fold
    Model_2.fit(PX_train_fold, Y_train_fold)
    
    # Predict on the validation fold and calculate the MSE
    Y_val_fold_pred_scaled = Model_2.predict(PX_val_fold)
    mse_fold_scaled = MSE(Y_val_fold, Y_val_fold_pred_scaled)
    mse_scaled_values.append(mse_fold_scaled)
    
    # Transform back from log scale and calculate the MSE
    Y_val_fold_pred = np.exp(Y_val_fold_pred_scaled)
    Y_val_fold_true = np.exp(Y_val_fold)
    mse_fold_true = MSE(Y_val_fold_true, Y_val_fold_pred)
    mse_true_values.append(mse_fold_true)

# Calculate the mean MSE across all folds
mean_mse_scaled = np.mean(mse_scaled_values)
mean_mse_true = np.mean(mse_true_values)

print(f'Mean MSE for scaled values across all folds: {mean_mse_scaled}')
print(f'Mean MSE for true values across all folds: {mean_mse_true}')

C:\Users\mi\AppData\Local\Temp\ipykernel_5020\2644733487.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  best_pca = parms.iloc[0][0]['pca__n_components']
C:\Users\mi\AppData\Local\Temp\ipykernel_5020\2644733487.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  best_ccp_alpha = parms.iloc[0][0]['RG__ccp_alpha']


Mean MSE for scaled values across all folds: 1.125328264383315
Mean MSE for true values across all folds: 1443.3420785293881


In [23]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np

# Load the target variable
OY_train = pd.read_csv("train_y_logsales.csv", index_col=0)

# Assuming X_train is already defined and holds the feature data

# Define the model
Model_3 = RandomForestRegressor(ccp_alpha=0.005, n_estimators=10)

# Prepare the cross-validation procedure
cv = KFold(n_splits=10, shuffle=True, random_state=1)

# Collect the MSE for each fold
mse_values = []
true_vs_predicted_mse_values = []

for train_ix, test_ix in cv.split(X_train):
    # Split the data
    X_train2, X_test2 = X_train.iloc[train_ix], X_train.iloc[test_ix]
    Y_train2, Y_test2 = OY_train.iloc[train_ix], OY_train.iloc[test_ix]

    # Fit the model on the training data
    Model_3.fit(X_train2, Y_train2.values.ravel())

    # Predict on the test data and calculate the MSE
    mse_value = mean_squared_error(Y_test2, Model_3.predict(X_test2))
    mse_values.append(mse_value)
    
    # Predict and transform back from log scale
    predicted_sales = np.exp(Model_3.predict(X_test2))
    true_sales = np.exp(Y_test2.values.ravel())

    # Calculate the MSE between the true and predicted sales
    final_mse = mean_squared_error(true_sales, predicted_sales)
    true_vs_predicted_mse_values.append(final_mse)

# Calculate the mean MSE across all folds
mean_mse = np.mean(mse_values)
mean_true_vs_predicted_mse = np.mean(true_vs_predicted_mse_values)

print(f'Mean MSE across all folds (scaled values): {mean_mse}')
print(f'Mean MSE between true and predicted sales across all folds: {mean_true_vs_predicted_mse}')


Mean MSE across all folds (scaled values): 0.011105743891110794
Mean MSE between true and predicted sales across all folds: 542.9825405551157


5.1 Evaluate Model original data



In [24]:

best_ccp_alpha = parms.iloc[0][0]['RG__ccp_alpha']
best_n_est = parms.iloc[0][0]['RG__n_estimators']
Model_2 = RandomForestRegressor(ccp_alpha=best_ccp_alpha,n_estimators=best_n_est)
X_train1,X_test1,Y_train1,Y_test1 = train_test_split(X_train,Y_train,test_size=0.2,random_state=132)
RG1 = Model_2
RG1.fit(X_train1,Y_train1.values.ravel())
MSE(RG1.predict(X_test1),Y_test1)

C:\Users\mi\AppData\Local\Temp\ipykernel_5020\3335317984.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  best_ccp_alpha = parms.iloc[0][0]['RG__ccp_alpha']
C:\Users\mi\AppData\Local\Temp\ipykernel_5020\3335317984.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  best_n_est = parms.iloc[0][0]['RG__n_estimators']


KeyError: 'RG__n_estimators'

In [ ]:

#I think here is wrong, how can it be the same

result = pd.DataFrame(Y_test1)
result["Predicted_sales"] = RG1.predict(X_test1)
MSE(result["sales"],result["Predicted_sales"])

In [ ]:

# Assuming best_ccp_alpha and best_n_est have been previously defined
Model_3 = RandomForestRegressor(ccp_alpha=best_ccp_alpha, n_estimators=best_n_est)

# Load the target variable, which is the log-transformed sales, from a CSV file
OY_train = pd.read_csv("train_y_logsales.csv", index_col=0)

# Assuming X_train is already defined and holds the feature data
# Split the data into training and testing sets
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X_train, OY_train, test_size=0.2, random_state=132)

# Fit the model on the training data
RG2 = Model_3
RG2.fit(X_train2, Y_train2.values.ravel())

# Predict on the test data
Y_test2_pred = RG2.predict(X_test2)

# Apply the exponential function to revert the log transformation for both predicted and actual values
Y_test2_pred_exp = np.exp(Y_test2_pred)  # Revert predictions
Y_test2_exp = np.exp(Y_test2.values.ravel())  # Revert actual values

# Calculate the Mean Squared Error on the original sales scale
mse_value = mean_squared_error(Y_test2_exp, Y_test2_pred_exp)
print(f'Mean Squared Error on test data (original sales scale): {mse_value}')